In [2]:
import numpy as np
import pandas as pd
import mne
from mne_bids import BIDSPath, read_raw_bids, get_entity_vals, get_datatypes, find_matching_paths
import xarray as xr
from ptsa.data.timeseries import TimeSeries
import os
import cmldask.CMLDask as da
import cmlreaders as cml


def create_xarray(ep):
    '''Create an xarray version of epoch data'''
    assert ep.metadata is not None, "Please define mne.Epoch.metadata"
    x = TimeSeries(ep.get_data(copy=True), dims=('event','channel','time'),
                   coords={'event':pd.MultiIndex.from_frame(ep.metadata),
                           'channel':ep.info['ch_names'],
                           'time':ep.times,
                           'samplerate':ep.info['sfreq']})
    return x

In [3]:
settings_path = "settings/NICLS_RO_encoding_features.pkl"
settings = da.Settings.Load(settings_path)
subject = 'LTP453'
data = cml.get_data_index(kind = 'ltp')
data = data[(data['experiment']==settings.experiment)&(data['subject']==subject)].sort_values('session').reset_index()

In [4]:
bids_root = "/data8/NICLS_BIDS"
bids_path = BIDSPath(subject=subject, root=bids_root)

In [5]:
bids_paths = find_matching_paths(
    bids_root, 
    subjects=subject, 
    tasks="NiclsCourierReadOnly",
    datatypes="eeg",
    extensions=".bdf"
    )
all_sessions = []
for bids_path in bids_paths[0:1]:
    events_path = bids_path.copy().update(suffix="events", extension=".tsv")
    event_df = pd.read_csv(events_path, sep="\t")
    raw = read_raw_bids(bids_path, extra_params={"infer_types":True}, verbose=False)
    mne_events, mne_event_id = mne.events_from_annotations(raw, verbose=False)
    epochs = mne.Epochs(
        raw,
        mne_events, 
        event_id={'WORD':mne_event_id['WORD']}, 
        event_repeated='drop', 
        tmin=(settings.rel_start-settings.buffer_time)/1000.,
        tmax=(settings.rel_stop+settings.buffer_time)/1000., 
        baseline=None, 
        preload=True,
        metadata=event_df # should allow querying with pandas syntax, but length has to match
    )
    all_sessions.append(create_xarray(epochs))
    

Adding metadata with 15 columns
120 matching events found
No baseline correction applied
0 projection items activated
Loading data for 120 events and 4097 original time points ...


/tmp/ipykernel_95104/3422786691.py:12: RuntimeWarning: The unit for channel(s) EXG5, EXG6, EXG7, EXG8 has changed from V to NA.
  raw = read_raw_bids(bids_path, extra_params={"infer_types":True}, verbose=False)
/tmp/ipykernel_95104/3422786691.py:12: RuntimeWarning: The unit for channel(s) Status has changed from NA to V.
  raw = read_raw_bids(bids_path, extra_params={"infer_types":True}, verbose=False)
/tmp/ipykernel_95104/3422786691.py:12: RuntimeWarning: There are channels without locations (n/a) that are not marked as bad: ['EXG1', 'EXG2', 'EXG3', 'EXG4', 'EXG5', 'EXG6', 'EXG7', 'EXG8', 'Status']
  raw = read_raw_bids(bids_path, extra_params={"infer_types":True}, verbose=False)
/tmp/ipykernel_95104/3422786691.py:12: RuntimeWarning: Not setting positions of 9 eog/misc/stim channels found in montage:
['EXG1', 'EXG2', 'EXG3', 'EXG4', 'EXG5', 'EXG6', 'EXG7', 'EXG8', 'Status']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling 

0 bad epochs dropped


: 

In [1]:
epochs

NameError: name 'epochs' is not defined

In [121]:
data = cml.get_data_index(kind = 'ltp')
data = data[(data['experiment']==settings.experiment)&(data['subject']==subject)].sort_values('session').reset_index()
# cml.CMLReader(subject=subject, experiment=row['experiment'], session=row['session'])
r = cml.CMLReader(subject, experiment=settings.experiment, session=0)
evs = r.load('task_events')
word_evs = evs[(evs.type=='WORD')&(evs.eegoffset!=-1)]
if len(word_evs)==0:
    # continue # sync pulses not recorded
    print("bad!")
eeg = r.load_eeg(word_evs,
                    rel_start=settings.rel_start - settings.buffer_time,
                    rel_stop=settings.rel_stop + settings.buffer_time,
                    clean=settings.clean
                ).to_ptsa()

Extracting EDF parameters from /protocols/ltp/subjects/LTP453/experiments/NiclsCourierReadOnly/sessions/3/ephys/current_processed/LTP453_session_3.bdf...
BDF file detected


Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 9517055  =      0.000 ...  4647.000 secs...
Not setting metadata
150 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 150 events and 4097 original time points ...
0 bad epochs dropped


In [127]:
epochs.get_data(copy=True)

array([[[ 0.01028528,  0.01028456,  0.01028337, ...,  0.0102895 ,
          0.01028912,  0.01028918],
        [ 0.00743881,  0.00743885,  0.00743781, ...,  0.00744356,
          0.00744222,  0.00744213],
        [ 0.00751331,  0.00751322,  0.00751381, ...,  0.00751856,
          0.00751685,  0.00751716],
        ...,
        [-0.25951622, -0.25951613, -0.259516  , ..., -0.25951544,
         -0.25951632, -0.2595161 ],
        [-0.25949075, -0.25948994, -0.25948985, ..., -0.25949107,
         -0.25949125, -0.25949088],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]],

       [[ 0.01029265,  0.01029218,  0.01029225, ...,  0.01029909,
          0.01029881,  0.01029597],
        [ 0.00745266,  0.00744947,  0.00744994, ...,  0.0074575 ,
          0.00745888,  0.0074581 ],
        [ 0.007523  ,  0.00752253,  0.00752494, ...,  0.00753475,
          0.00753453,  0.00753106],
        ...,
        [-0.25951535, -0.2595161 , -0.25951697, ..., -

In [122]:
eeg

<xarray.TimeSeries (event: 150, channel: 137, time: 4097)>
array([[[ 0.01028528,  0.01028456,  0.01028337, ...,  0.0102895 ,
          0.01028912,  0.01028918],
        [ 0.00743881,  0.00743885,  0.00743781, ...,  0.00744356,
          0.00744222,  0.00744213],
        [ 0.00751331,  0.00751322,  0.00751381, ...,  0.00751856,
          0.00751685,  0.00751716],
        ...,
        [-0.25951622, -0.25951613, -0.259516  , ..., -0.25951544,
         -0.25951632, -0.2595161 ],
        [-0.25949075, -0.25948994, -0.25948985, ..., -0.25949107,
         -0.25949125, -0.25949088],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]],

       [[ 0.01029265,  0.01029218,  0.01029225, ...,  0.01029909,
          0.01029881,  0.01029597],
        [ 0.00745266,  0.00744947,  0.00744994, ...,  0.0074575 ,
          0.00745888,  0.0074581 ],
        [ 0.007523  ,  0.00752253,  0.00752494, ...,  0.00753475,
          0.00753453,  0.00753106],
...
        [-0.25949119, -0.25948982, -0.25948888, ..., -0.2594875 ,
         -0.25948841, -0.25949047],
        [-0.25946572, -0.25946566, -0.25946638, ..., -0.25946497,
         -0.25946494, -0.25946507],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]],

       [[ 0.00873794,  0.00873816,  0.00873897, ...,  0.00874466,
          0.00874447,  0.00874584],
        [ 0.00649725,  0.00649566,  0.00649619, ...,  0.00650235,
          0.0065036 ,  0.00650491],
        [ 0.00725297,  0.00725525,  0.00725728, ...,  0.00726216,
          0.00726141,  0.00725953],
        ...,
        [-0.25948994, -0.25948941, -0.259488  , ..., -0.25948935,
         -0.25948985, -0.2594916 ],
        [-0.25946429, -0.25946229, -0.25946432, ..., -0.2594661 ,
         -0.25946591, -0.25946613],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]]])
Coordinates:
  * event                       (event) MultiIndex
  - classifier                  (event) object 'X' 'X' 'X' 'X' ... 'X' 'X' 'X'
  - correctPointingDirection    (event) int64 -999 -999 -999 ... -999 -999 -999
  - eegfile                     (event) object '/protocols/ltp/subjects/LTP45...
  - eegoffset                   (event) int64 567722 584140 ... 8776272 8789958
  - efr_mark                    (event) int64 -999 -999 -999 ... -999 -999 -999
  - eogArtifact                 (event) int64 0 0 1 0 0 0 0 0 ... 0 0 0 0 1 0 0
  - experiment                  (event) object 'NiclsCourierReadOnly' ... 'Ni...
  - finalrecalled               (event) int64 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0
  - intruded                    (event) int64 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0
  - intrusion                   (event) int64 -999 -999 -999 ... -999 -999 -999
  - item                        (event) object 'BREAD' 'SUIT' ... 'HAIR_SPRAY'
  - itemno                      (event) int64 -999 -999 -999 ... -999 -999 -999
  - montage                     (event) int64 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0
  - msoffset                    (event) int64 -1 -1 -1 -1 -1 ... -1 -1 -1 -1 -1
  - mstime                      (event) int64 1626356086713 ... 1626360101813
  - phase                       (event) object '1' '1' '1' '1' ... '2' '2' '2'
  - presX                       (event) float64 37.59 49.19 ... -28.33 9.383
  - presZ                       (event) float64 49.66 25.53 ... -45.97 -49.84
  - protocol                    (event) object 'ltp' 'ltp' 'ltp' ... 'ltp' 'ltp'
  - recalled                    (event) int64 1 1 0 1 1 1 1 1 ... 1 0 0 1 1 1 1
  - rectime                     (event) int64 -999 -999 -999 ... -999 -999 -999
  - serialpos                   (event) int64 1 2 3 4 5 6 ... 10 11 12 13 14 15
  - session                     (event) int64 3 3 3 3 3 3 3 3 ... 3 3 3 3 3 3 3
  - store                       (event) object 'bakery' ... 'barber_shop'
  - storeX                      (event) float64 36.69 58.31 ... -29.59 14.6
  - storeZ            